In [3]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
import geopandas as gpd
import pandas as pd
import PIL
from pathlib import Path
import tqdm
import rasterio.features
import shapely.geometry
import glob

# COCO tools
import pycocotools.mask
from pycocotools.coco import COCO

# # Recognize Anything Model & Tag2Text
# from ram.models import ram_plus
# from ram import inference_ram
# import torchvision.transforms as TS

# Grounding Dino
from groundingdino.util.inference import load_model
import groundingdino.datasets.transforms as T
from groundingdino.util.utils import get_phrases_from_posmap

# Segment anything
from segment_anything_hq import (
    SamPredictor as SamPredictor_hq,
    sam_model_registry as sam_model_registry_hq,
)
from segment_anything import SamPredictor, sam_model_registry

/opt/tljh/user/envs/opschotdetectie2/lib/python3.11/site-packages/segment_anything_hq/modeling/tiny_vit_sam.py:662: UserWarning: Overwriting tiny_vit_5m_224 in registry with segment_anything_hq.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/opt/tljh/user/envs/opschotdetectie2/lib/python3.11/site-packages/segment_anything_hq/modeling/tiny_vit_sam.py:662: UserWarning: Overwriting tiny_vit_11m_224 in registry with segment_anything_hq.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/opt/tljh/user/envs/opschotdetectie2/lib/python3.11/site-packages/segment_anything_hq/modeling/tiny_vit_sam.py:662: UserWarning: Overwriting tiny_vit_21m_224 in registry with segment_anything_hq.modeling.tiny_vit_sam.tiny_vit_21m_224.

In [4]:
from hydra import initialize, compose

with initialize("config", version_base=None):
    cfg = compose("config.yaml")

print(cfg)
# Load grounding dino model
dino_model = load_model(
    cfg.GROUNDING_DINO_CONFIG_PATH,
    cfg.GROUNDING_DINO_CHECKPOINT_PATH,
    device=cfg.DEVICE,
)

# Segment Anything Model (SAM)

if cfg.USE_SAM_HQ:
    print("Initialize SAM-HQ Predictor")
    sam = sam_model_registry_hq[cfg.SAM_HQ_ENCODER_VERSION](
        checkpoint=cfg.SAM_HQ_CHECKPOINT_PATH
    ).to(device=cfg.DEVICE)
    sam_predictor = SamPredictor_hq(sam)
else:
    sam = sam_model_registry[cfg.SAM_ENCODER_VERSION](
        checkpoint=cfg.SAM_CHECKPOINT_PATH
    ).to(device="cuda:1")
    sam_predictor = SamPredictor(sam)

{'device': 'cuda:1', 'hq': False, 'model_type': 'vit_b', 'disk_path': './', 'data_path': '${disk_path}/data', 'checkpoint': 'sam_vit_b_01ec64.pth', 'DEVICE': 'cuda:1', 'DINO_BOX_THRESHOLD': 0.2, 'DINO_TEXT_THRESHOLD': 0.0, 'DO_IOU_MERGE': True, 'IOU_THRESHOLD': 0.5, 'dataDir': '${data_path}/supervisely/277426_20230714 MUG Hoorn Enkhuizen orthomosaic deel 6/2023-12-11 12_52_14.810', 'path_tifftiles': '${data_path}/label_data', 'use_fixed_tags': True, 'RAM_CHECKPOINT': '${disk_path}/ram_weights/ram_plus_swin_large_14m.pth', 'RAM_MODELTYPE': 'swin_l', 'GROUNDING_DINO_CONFIG_PATH': 'config/GroundingDINO_SwinB_cfg.py', 'GROUNDING_DINO_CHECKPOINT_PATH': '${disk_path}/dino_weights/groundingdino_swinb_cogcoor.pth', 'USE_SAM_HQ': False, 'SAM_ENCODER_VERSION': 'vit_h', 'SAM_CHECKPOINT_PATH': '${disk_path}/sam_weights/sam_vit_h_4b8939.pth', 'SAM_HQ_ENCODER_VERSION': 'vit_h', 'SAM_HQ_CHECKPOINT_PATH': '${disk_path}/sam_weights/sam_hq_vit_h.pth', 'fixed_tags': ['bush', 'grass', 'road', 'shrub', 'st

/opt/tljh/user/envs/opschotdetectie2/lib/python3.11/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025842427/work/aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


In [6]:
def find_tile_bounds(root_tilepath, concat=True):
    """
    Recursively search through directories starting from root_tilepath to find and optionally concatenate
    geospatial data files (.gpkg) that include 'cells_intersect' in their filename.

    Parameters:
    - root_tilepath (str or pathlib.Path): The root directory path where the search for tile files begins.
    - concat (bool, optional): If True, concatenates all found geospatial data into a single DataFrame.
                               If False, returns a list of DataFrames. Default is True.

    Returns:
    - pandas.DataFrame or list of geopandas.GeoDataFrame: The concatenated DataFrame of all files if `concat=True`,
      or a list of DataFrames for each file if `concat=False`.

    """
    cell_files = []
    for p in Path(str(root_tilepath)).iterdir():
        if p.is_dir():
            cell_files += find_tile_bounds(p, concat=False)
        elif p.is_file() and p.suffix == ".gpkg" and "tiles_intersect" in p.stem:
            df = gpd.read_file(p)
            df["name"] = p.stem.replace("_tiles_intersects", "")
            cell_files.append(df.copy())

    if concat:
        cell_files = pd.concat(cell_files)

    return cell_files


# tilebounds
df_tilebounds = find_tile_bounds(Path(cfg.disk_path) / "tile_dataset_havenhoofden")
df_tilebounds

,value,xmin,ymin,xmax,ymax,geometry,name
0,228.0,135960.411490,541410.562577,135975.920790,541426.418838,"POLYGON ((135960.411 541426.419, 135960.411 54...",20230714 MUG Medemblik Den Oever orthomosaic d...
1,1313.0,135975.920790,541410.562577,135991.430090,541426.418838,"POLYGON ((135975.921 541426.419, 135975.921 54...",20230714 MUG Medemblik Den Oever orthomosaic d...
2,934.0,135960.411490,541394.706317,135975.920790,541410.562577,"POLYGON ((135960.411 541410.563, 135960.411 54...",20230714 MUG Medemblik Den Oever orthomosaic d...
3,1184.0,135975.920790,541394.706317,135991.430090,541410.562577,"POLYGON ((135975.921 541410.563, 135975.921 54...",20230714 MUG Medemblik Den Oever orthomosaic d...
4,1406.0,135960.411490,541378.850056,135975.920790,541394.706317,"POLYGON ((135960.411 541394.706, 135960.411 54...",20230714 MUG Medemblik Den Oever orthomosaic d...
...,...,...,...,...,...,...,...
15,1275.0,132311.157233,549165.308178,132325.792556,549180.157760,"POLYGON ((132311.157 549180.158, 132311.157 54...",20230714 MUG Medemblik Den Oever orthomosaic d...
16,5450.0,132325.792556,549165.308178,132340.427879,549180.157760,"POLYGON ((132325.793 549180.158, 132325.793 54...",20230714 MUG Medemblik Den Oever orthomosaic d...
17,1935.0,132311.157233,549150.458597,132325.792556,549165.308178,"POLYGON ((132311.157 549165.308, 132311.157 54...",20230714 MUG Medemblik Den Oever orthomosaic d...
18,5191.0,132325.792556,549150.458597,132340.427879,549165.308178,"POLYGON ((132325.793 549165.308, 132325.793 54...",20230714 MUG Medemblik Den Oever orthomosaic d...


In [7]:
out_dir = Path(cfg.disk_path) / "output_havenhoofden"
if not out_dir.exists():
    out_dir.mkdir()
else:
    print(f"Directory {out_dir} already exists!")

In [8]:
Path(cfg.disk_path) / "tile_dataset_havenhoofden"

PosixPath('data/tile_dataset_havenhoofden')

In [9]:
dataDirs = [
    Path(p)
    for p in glob.glob(
        str(
            Path(cfg.disk_path)
            / "tile_dataset_havenhoofden"
            / "20230714 MUG Medemblik Den Oever orthomosaic deel *"
        )
    )
]

for dataDir in dataDirs:
    print(dataDir)

    projName = dataDir.stem

    df_pred_shapes = dict(
        category=[],
        confidence=[],
        tile_path=[],
        project_name=[],
        tile_fname=[],
        geometry=[],
    )

    imgPaths = glob.glob(str(dataDir / "tiles_havenhoofden" / "*.jpeg"))

    for imgPath in tqdm.tqdm(imgPaths):
        print(imgPath)

        # Load image
        image_pil, image = load_image(imgPath)

        # Tags
        if cfg.fixed_tags:
            tags = ",".join(cfg.fixed_tags)
        else:
            # Find tags with RAM
            ram_model = ram_model.to(cfg.DEVICE)
            raw_image = image_pil.resize((384, 384))
            raw_image = transform(raw_image).unsqueeze(0).to(cfg.DEVICE)
            res = inference_ram(raw_image, ram_model)
            tags = res[0].replace(" |", ",")

        # Find bounding boxes with grounding dino
        boxes_filt, scores, pred_phrases = get_grounding_output(
            dino_model,
            image,
            tags,
            DINO_BOX_THRESHOLD,
            DINO_TEXT_THRESHOLD,
            device=DEVICE,
        )

        # Resize boxes
        size = image_pil.size
        H, W = size[1], size[0]
        for i in range(boxes_filt.size(0)):
            boxes_filt[i] = boxes_filt[i] * torch.Tensor([W, H, W, H])
            boxes_filt[i][:2] -= boxes_filt[i][2:] / 2
            boxes_filt[i][2:] += boxes_filt[i][:2]

        # use NMS to handle overlapped boxes
        boxes_filt = boxes_filt.cpu()
        nms_idx = (
            torchvision.ops.nms(boxes_filt, scores, IOU_THRESHOLD).numpy().tolist()
        )
        if DO_IOU_MERGE:
            boxes_filt_clean = boxes_filt[nms_idx]
            pred_phrases_clean = [pred_phrases[idx] for idx in nms_idx]
            # print(f"NMS: before {boxes_filt.shape[0]} boxes, after {boxes_filt_clean.shape[0]} boxes")
        else:
            boxes_filt_clean = boxes_filt
            pred_phrases_clean = pred_phrases

        # Segment objects with SAM
        image_np = np.array(image_pil)
        sam_predictor.set_image(image_np)
        transformed_boxes = sam_predictor.transform.apply_boxes_torch(
            boxes_filt_clean, image_np.shape[:2]
        ).to(DEVICE)
        masks, _, _ = sam_predictor.predict_torch(
            point_coords=None,
            point_labels=None,
            boxes=transformed_boxes.to(DEVICE),
            multimask_output=False,
        )

        # Find tile bounds (X, Y) based on name
        imgName = Path(imgPath).stem
        tile1a = imgName.split("_")[0]
        tile1b = int(imgName.split("_")[-1])
        cellfile = df_tilebounds[
            (df_tilebounds.index == tile1b) & (df_tilebounds.name == tile1a)
        ].copy()
        assert len(cellfile) == 1
        cellfile = cellfile.iloc[0, :].copy()
        xstep = (cellfile.xmax - cellfile.xmin) / image_np.shape[1]
        ystep = (cellfile.ymax - cellfile.ymin) / image_np.shape[0]

        # eventueel nog geometry van cellfile ipv tabel,
        # eventueel test via inladen tiff

        affine = [xstep, 0, cellfile.xmin, 0, -ystep, cellfile.ymax, 0, 0, 1]

        # SAM masks
        assert len(pred_phrases_clean) == len(masks)
        shapes, titles = [], []
        for cat_title, mask in zip(pred_phrases_clean, masks):
            mask = mask.cpu().numpy()
            cat_shapes = rasterio.features.shapes(
                mask.astype(np.uint8), mask=mask, connectivity=4, transform=affine
            )
            for shape, _ in cat_shapes:
                title, confidence = cat_title.replace(")", "").split("(")
                shape = shapely.geometry.shape(shape).simplify(
                    0.01, preserve_topology=True
                )
                if shape.area > 0.01:
                    df_pred_shapes["category"].append(title)
                    df_pred_shapes["confidence"].append(confidence)
                    df_pred_shapes["geometry"].append(shape)

                    df_pred_shapes["tile_path"].append(str(imgPath))
                    df_pred_shapes["tile_fname"].append(Path(imgPath).stem)
                    df_pred_shapes["project_name"].append(projName)

    df_pred_shapes = gpd.GeoDataFrame(df_pred_shapes, crs="epsg:28992")
    if USE_SAM_HQ:
        df_pred_shapes.to_file(out_dir / f"fix_tags_hq_{projName}_havenhoofden.gpkg")
    else:
        df_pred_shapes.to_file(out_dir / f"fix_tags_{projName}_havenhoofden.gpkg")

data/tile_dataset_havenhoofden/20230714 MUG Medemblik Den Oever orthomosaic deel 8


  0%|          | 0/55 [00:00<?, ?it/s]

data/tile_dataset_havenhoofden/20230714 MUG Medemblik Den Oever orthomosaic deel 8/tiles_havenhoofden/20230714 MUG Medemblik Den Oever orthomosaic deel 8_0.jpeg


NameError: name 'load_image' is not defined